# Creating a Neural Network to classify diabetic patients

<img src="https://www.ndr.de/diabetes312_v-fullhd.jpg" align="left" width="600">

## Libraries and settings

In [ ]:
# Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prettytable import PrettyTable

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

<h3>The Problem</h3>
<p>Suppose we have some information about obesity, smoking habits, and exercise habits of five people. We also know whether these people are diabetic or not. The dataset looks like this:</p>
<table class="table table-striped">
    <thead>
        <tr>
            <th>Person</th>
            <th>Smoking</th>
            <th>Obesity</th>
            <th>Exercise</th>
            <th>Diabetic</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Person 1</td>
            <td>0</td>
            <td>1</td>
            <td>0</td>
            <td>1</td>
        </tr>
        <tr>
            <td>Person 2</td>
            <td>0</td>
            <td>0</td>
            <td>1</td>
            <td>0</td>
        </tr>
        <tr>
            <td>Person 3</td>
            <td>1</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
        </tr>
        <tr>
            <td>Person 4</td>
            <td>1</td>
            <td>1</td>
            <td>0</td>
            <td>1</td>
        </tr>
        <tr>
            <td>Person 5</td>
            <td>1</td>
            <td>1</td>
            <td>1</td>
            <td>1</td>
        </tr>
    </tbody>
</table>
<p>In this table, there are five columns: Person, Smoking, Obesity, Exercise, and Diabetic. Here 1 refers to true and 0 refers to false. For instance, the first person has values of 0, 1, 0 which means that the person doesn't smoke, is obese, and doesn't exercise. The person is also diabetic. It is clearly evident from the dataset that a person's obesity is indicative of him being diabetic. Our task is to create a neural network that is able to predict whether an unknown person is diabetic or not given data about his exercise habits, obesity, and smoking habits.</p>

In [2]:
# Function to generate pretty tables
def generate_ascii_table(df):
    x = PrettyTable()
    x.field_names = df.columns.tolist()
    for row in df.values:
        x.add_row(row)
    print(x)
    return x

# Activation function (sigmoid)
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivative of the activation function
# -> When updating the curve, to know in which 
#    direction and how much to change or update 
#    the curve depending upon the slope.
def sigmoid_der(x):
    return sigmoid(x)*(1-sigmoid(x))

In [ ]:
# Create data frame
data = np.array([[0,1,0,1],[0,0,1,0],[1,0,0,0],[1,1,0,1],[1,1,1,1]])
df = pd.DataFrame(data, columns=['Smoking', 'Obesity', 'Exercise', 'Diabetic'])

# Show table
generate_ascii_table(df)

# Matrix with features (X) and labels (y)
X = data[0:5,0:3]
y = data[0:5,3]
y = y.reshape(5,1)

In [ ]:
# Plots the sigmoid function and its derivative
input = np.linspace(-10, 10, 100)
plt.plot(input, sigmoid(input), color="green")
plt.plot(input, sigmoid(input)*(1-sigmoid(input)), color="orange")
plt.grid(color='gray', linestyle='-', linewidth=0.1)

In [5]:
# Initialize weights and bias & define learning rate und the number of epochs
np.random.seed(42)
weights        = np.random.rand(3,1)
bias           = np.random.rand(1)
learning_rate  = 0.25
num_epochs     = 200

In [ ]:
# Calculations step by step

# Inputs
inputs = X

# Print X, y, weights and bias
print('X:', '\n', inputs, '\n')
print('y:', '\n', y, '\n')
print('weights:', '\n', weights,'\n')
print('bias:', '\n', bias,'\n')

# Calculate X * weigths + bias
XW = np.dot(X, weights) + bias
print('XW:', '\n', XW, '\n')

# Apply activation function to XW
z = sigmoid(XW)
print('sigmoid(XW) = z', '\n', z, '\n')

# Calculate the error (difference between labels and z)
error = z - y
print('error:', '\n', error, '\n')

# Backpropagation (weights and bias adaptation)
dcost_dpred = error
dpred_dz    = sigmoid_der(z)
print('sigmoid_der(z):', '\n', dpred_dz, '\n')

# Calculate
z_delta = dcost_dpred * dpred_dz
print('z_delta:', '\n', z_delta, '\n')

# Adapt weights
inputs = X.T
weights -= learning_rate * np.dot(inputs, z_delta)
print('adapted weights', '\n', weights, '\n')

print('adapted bias:')
for num in z_delta:
    bias -= learning_rate * num
    print(bias)

In [ ]:
# Neural network

# Prepare figure
fig = plt.figure()
ax  = fig.add_subplot(111)
ax.set_xlabel('Epoche')
ax.set_ylabel('Error')
plt.title('Error per Epoche')
plt.ion()

fig.show()
fig.canvas.draw()

#--------------------------------------
# Loop with backpropagation
#--------------------------------------

# Initialize list
d = []

for epoch in range(num_epochs):
    
    inputs = X
    
    # Calculate (X * weigths) + bias
    XW = np.dot(X, weights) + bias

    # Apply activation function to XW
    z = sigmoid(XW)

    # Calculate the error (difference between label-value and z)
    error = z - y
    
    # Store error in data frame
    d.append(
        {
            'Epoch': epoch,
            'Error': error.sum()
        }
    )
    df = pd.DataFrame(d)

    # Backpropagation (change weights and bias)
    dcost_dpred = error
    dpred_dz    = sigmoid_der(z)
    z_delta     = dcost_dpred * dpred_dz

    inputs      = X.T
    weights    -= learning_rate * np.dot(inputs, z_delta)

    for num in z_delta:
        bias -= learning_rate * num

    #-----------------------
    # Plot error per epoche
    #-----------------------
    plt.xlim([0,num_epochs])
    plt.ylim([0, df.Error[0]])
    ax.plot(df.Epoch, df.Error)
    fig.canvas.draw()

In [ ]:
# Model-prediction based on new data
data_new = np.array([[0,1,0]])
df_new   = pd.DataFrame(data_new, columns=['Smoking', 'Obesity', 'Exercise'])

print('New Data (basis for prediction):', '\n')
generate_ascii_table(df_new)

# Calculate probability of having Diabetes
result       = sigmoid(np.dot(data_new, weights) + bias)
print('\n', 'Predicted probability of having Diabetes:', result)

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')